# layers

Type an introduction of the package here.

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.layers import * 

In [ ]:
show_doc(AdaptiveConcatPool2d)

### <a id=AdaptiveConcatPool2d></a><em>class</em> `AdaptiveConcatPool2d`
(<code>sz</code>:`Optional`[<code>int</code>]=`None`) :: [<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L59">[source]</a></div>


Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d`

[`AdaptiveConcatPool2d`](/layers.html#AdaptiveConcatPool2d)

In [ ]:
show_doc(AdaptiveConcatPool2d.forward)

#### <a id=forward></a>`forward`
(<code>x</code>)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L66">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`AdaptiveConcatPool2d.forward`

In [ ]:
show_doc(CrossEntropyFlat)

### <a id=CrossEntropyFlat></a><em>class</em> `CrossEntropyFlat`
(<code>weight</code>=`None`, <code>size_average</code>=`None`, <code>ignore_index</code>=`-100`, <code>reduce</code>=`None`, <code>reduction</code>=`'elementwise_mean'`) :: [<code>CrossEntropyLoss</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L92">[source]</a></div>


Same as `nn.CrossEntropyLoss`, but flattens input and target

[`CrossEntropyFlat`](/layers.html#CrossEntropyFlat)

In [ ]:
show_doc(CrossEntropyFlat.forward)

#### <a id=forward></a>`forward`
(<code>input</code>:<code>Tensor</code>, <code>target</code>:<code>Tensor</code>) -> <code>OneEltTensor</code><div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L94">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`CrossEntropyFlat.forward`

In [ ]:
show_doc(Debugger)

### <a id=Debugger></a><em>class</em> `Debugger`
() :: [<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L68">[source]</a></div>


A module to debug inside a model

[`Debugger`](/layers.html#Debugger)

In [ ]:
show_doc(Debugger.forward)

#### <a id=forward></a>`forward`
(<code>x</code>:<code>Tensor</code>) -> <code>Tensor</code><div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L70">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`Debugger.forward`

In [ ]:
show_doc(Flatten)

#### <a id=Flatten></a>`Flatten`
() -> <code>Tensor</code><div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L20">[source]</a></div>


Flattens `x` to a single dimension, often used at the end of a model

[`Flatten`](/layers.html#Flatten)

In [ ]:
show_doc(Lambda)

### <a id=Lambda></a><em>class</em> `Lambda`
(<code>func</code>:<code>Callable</code>[<code>Tensor</code>, <code>Tensor</code>]) :: [<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L7">[source]</a></div>


An easy way to create a pytorch layer for a simple `func`

[`Lambda`](/layers.html#Lambda)

In [ ]:
show_doc(Lambda.forward)

#### <a id=forward></a>`forward`
(<code>x</code>)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L14">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`Lambda.forward`

In [ ]:
show_doc(PoolFlatten)

#### <a id=PoolFlatten></a>`PoolFlatten`
() -> [<code>Sequential</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L24">[source]</a></div>


Apply `nn.AdaptiveAvgPool2d` to `x` and then flatten the result

[`PoolFlatten`](/layers.html#PoolFlatten)

In [ ]:
show_doc(ResizeBatch)

#### <a id=ResizeBatch></a>`ResizeBatch`
(<code>size</code>:<code>int</code>) -> <code>Tensor</code><div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L16">[source]</a></div>


Layer that resizes x to `size`, good for connecting mismatched layers

[`ResizeBatch`](/layers.html#ResizeBatch)

In [ ]:
show_doc(StdUpsample)

### <a id=StdUpsample></a><em>class</em> `StdUpsample`
(<code>n_in</code>:<code>int</code>, <code>n_out</code>:<code>int</code>) :: [<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L74">[source]</a></div>


Standard upsample module

[`StdUpsample`](/layers.html#StdUpsample)

In [ ]:
show_doc(StdUpsample.forward)

#### <a id=forward></a>`forward`
(<code>x</code>:<code>Tensor</code>) -> <code>Tensor</code><div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L81">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`StdUpsample.forward`

In [ ]:
show_doc(bn_drop_lin)

#### <a id=bn_drop_lin></a>`bn_drop_lin`
(<code>n_in</code>:<code>int</code>, <code>n_out</code>:<code>int</code>, <code>bn</code>:<code>bool</code>=`True`, <code>p</code>:<code>float</code>=`0.0`, <code>actn</code>:`Optional`[[<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)]=`None`)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L28">[source]</a></div>


`n_in`->bn->dropout->linear(`n_in`,`n_out`)->`actn`

[`bn_drop_lin`](/layers.html#bn_drop_lin)

In [ ]:
show_doc(conv2d)

#### <a id=conv2d></a>`conv2d`
(<code>ni</code>:<code>int</code>, <code>nf</code>:<code>int</code>, <code>ks</code>:<code>int</code>=`3`, <code>stride</code>:<code>int</code>=`1`, <code>padding</code>:<code>int</code>=`None`, <code>bias</code>=`False`) -> [<code>Conv2d</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L36">[source]</a></div>


Create `nn.Conv2d` layer: `ni` inputs, `nf` outputs, `ks` kernel size. `padding` defaults to `k//2`

[`conv2d`](/layers.html#conv2d)

In [ ]:
show_doc(conv2d_relu)

#### <a id=conv2d_relu></a>`conv2d_relu`
(<code>ni</code>:<code>int</code>, <code>nf</code>:<code>int</code>, <code>ks</code>:<code>int</code>=`3`, <code>stride</code>:<code>int</code>=`1`, <code>padding</code>:<code>int</code>=`None`, <code>bn</code>:<code>bool</code>=`False`) -> [<code>Sequential</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L48">[source]</a></div>


Create a [<code>conv2d</code>](http://docs.fast.ai/layers.html#conv2d) layer with `nn.ReLU` activation and optional(`bn`) `nn.BatchNorm2d`

[`conv2d_relu`](/layers.html#conv2d_relu)

In [ ]:
show_doc(conv2d_trans)

#### <a id=conv2d_trans></a>`conv2d_trans`
(<code>ni</code>:<code>int</code>, <code>nf</code>:<code>int</code>, <code>ks</code>:<code>int</code>=`2`, <code>stride</code>:<code>int</code>=`2`, <code>padding</code>:<code>int</code>=`0`) -> [<code>ConvTranspose2d</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.ConvTranspose2d)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L55">[source]</a></div>


Create `nn.nn.ConvTranspose2d` layer: `ni` inputs, `nf` outputs, `ks` kernel size. `padding` defaults to 0

[`conv2d_trans`](/layers.html#conv2d_trans)

In [ ]:
show_doc(conv_layer)

#### <a id=conv_layer></a>`conv_layer`
(<code>ni</code>:<code>int</code>, <code>nf</code>:<code>int</code>, <code>ks</code>:<code>int</code>=`3`, <code>stride</code>:<code>int</code>=`1`) -> [<code>Sequential</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L41">[source]</a></div>


Create Conv2d->BatchNorm2d->LeakyReLu layer: `ni` input, `nf` out filters, `ks` kernel, `stride`:stride

[`conv_layer`](/layers.html#conv_layer)

In [ ]:
show_doc(get_embedding)

#### <a id=get_embedding></a>`get_embedding`
(<code>ni</code>:<code>int</code>, <code>nf</code>:<code>int</code>) -> [<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L111">[source]</a></div>


Creates an embedding layer

[`get_embedding`](/layers.html#get_embedding)

In [ ]:
show_doc(simple_cnn)

#### <a id=simple_cnn></a>`simple_cnn`
(<code>actns</code>:<code>Collection</code>[<code>int</code>], <code>kernel_szs</code>:<code>Collection</code>[<code>int</code>], <code>strides</code>:<code>Collection</code>[<code>int</code>]) -> [<code>Sequential</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L98">[source]</a></div>


CNN with [<code>conv2d_relu</code>](http://docs.fast.ai/layers.html#conv2d_relu) layers defined by `actns`, `kernel_szs` and `strides`

[`simple_cnn`](/layers.html#simple_cnn)

In [ ]:
show_doc(std_upsample_head)

#### <a id=std_upsample_head></a>`std_upsample_head`
(<code>c</code>, <code>nfs</code>:<code>Collection</code>[<code>int</code>]) -> [<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L84">[source]</a></div>


Creates a sequence of upsample layers

[`std_upsample_head`](/layers.html#std_upsample_head)

In [ ]:
show_doc(trunc_normal_)

#### <a id=trunc_normal_></a>`trunc_normal_`
(<code>x</code>:<code>Tensor</code>, <code>mean</code>:<code>float</code>=`0.0`, <code>std</code>:<code>float</code>=`1.0`) -> <code>Tensor</code><div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/layers.py#L106">[source]</a></div>


Truncated normal initialization

[`trunc_normal_`](/layers.html#trunc_normal_)